# Part 1 Linear Programming

In [1]:
import numpy as np
import math

In [2]:
def simplex(c1, c2, a11, a12, a21, a22, b1, b2):
    # Create the tableau
    tableau = np.array([[a11, a12, 1, 0, b1],
                        [a21, a22, 0, 1, b2],
                        [-c1, -c2, 0, 0, 0]], dtype=float)

    # Perform the simplex algorithm
    while True:
        # Check if the solution is optimal
        if tableau[2, :-1].min() >= 0:
            break
        
        # Pick the starting variable
        s = np.argmin(tableau[2, :-1])
        
        # Check if the problem is unbounded
        if tableau[:, s].max() <= 0:
            return 'Unbounded'
        
        # Pick the leaving variable
        i = int(np.argmin(tableau[:-1, -1] / tableau[:-1, s]))
        
        # Update the tableau
        tableau[i, :] = tableau[i, :] / tableau[i, s]
        for k in range(tableau.shape[0]):
            if k != i:
                tableau[k, :] = tableau[k, :] - tableau[k, s] * tableau[i, :]

    z = tableau[-1, -1]
    x1 = tableau[0, -1]
    x2 = tableau[1, -1]

    return z, x1, x2

In [3]:
z, x1, x2= simplex(c1=100, c2=150, a11=3, a12=2, a21=1, a22=2, b1=6, b2=5)
print(f"Optimal solution: z={z}, x1={x1}, x2={x2}")

Optimal solution: z=387.5, x1=0.5, x2=2.25


# Part 2 Integer Programming

In [4]:
def integer(c1, c2, a11, a12, a21, a22, b1, b2):
    # Create the tableau
    tableau = np.array([[a11, a12, 1, 0, b1],
                        [a21, a22, 0, 1, b2],
                        [-c1, -c2, 0, 0, 0]], dtype=float)

    # Perform the simplex algorithm
    while True:
        # Check if the solution is optimal
        if tableau[2, :-1].min() >= 0:
            break
        
        # Pick the starting variable
        s = np.argmin(tableau[2, :-1])
        
        # Check if the problem is unbounded
        if tableau[:, s].max() <= 0:
            return 'Unbounded'
        
        # Pick the leaving variable
        i = int(np.argmin(tableau[:-1, -1] / tableau[:-1, s]))
        
        # Update the tableau
        tableau[i, :] = tableau[i, :] / tableau[i, s]
        for k in range(tableau.shape[0]):
            if k != i:
                tableau[k, :] = tableau[k, :] - tableau[k, s] * tableau[i, :]

   
    z = tableau[-1, -1]
    x1 = tableau[0, -1]
    x2 = tableau[1, -1]


    #INTEGER PROGRAMMING

    #set variables to 0
    z1=0
    z2=0
    z3=0
    z4=0

    #get upper limit of x1
    x1up = math.ceil(x1)
    #solve constraint 1 and 2 and compare results
    x2a = (b1 - a11*x1up) / a12
    x2b = (b2 - a22*x1up) / a22
    if x2a >= 0 and x2b >= 0:
        # get the lower result
        x2c = min([x2a, x2b])
        #branch off to upper and lower limit of x2
        x2c_up = math.ceil(x2c)
        x2c_dn = math.floor(x2c)
        #test upper limit against 2 contraints and record z value if passed
        if a11*x1up + a12*x2c_up <= b1 and a21*x1up + a22*x2c_up <= b2:
            z1 = c1*x1up + c2*x2c_up
        if a11*x1up + a12*x2c_dn <= b1 and a21*x1up + a22*x2c_dn <= b2:
            z2 = c1*x1up + c2*x2c_dn

    #get lower limit of x1
    x1dn = math.floor(x1)
    #solve constraint 1 and 2 and compare results
    x2d = (b1 - a11*x1dn) / a12
    x2e = (b2 - a22*x1dn) / a22
    #test against positivity constraint
    if x2d >= 0 and x2e >= 0:
        # get the lower result
        x2f = min([x2d, x2e])
        #branch off to upper and lower limit of x2
        x2f_up = math.ceil(x2f)
        x2f_dn = math.floor(x2f)
        #test upper limit against 2 contraints and record z value if passed
        if a11*x1dn + a12*x2f_up <= b1 and a21*x1dn + a22*x2f_up <= b2:
            z3 = c1*x1dn + c2*x2f_up
        if a11*x1dn + a12*x2f_dn <= b1 and a21*x1dn + a22*x2f_dn <= b2:
            z4 = c1*x1dn + c2*x2f_dn
    #return the node with best z value
    if max([z1,z2,z3,z4])==z1:
        z = z1
        x1 = x1up
        x2 = x2c_up
    elif max([z1,z2,z3,z4])==z2:
        z = z2
        x1 = x1up
        x2 = x2c_dn
    elif max([z1,z2,z3,z4])==z3:
        z = z3
        x1 = x1dn
        x2 = x2f_up
    elif max([z1,z2,z3,z4])==z4:
        z = z4
        x1 = x1dn
        x2 = x2f_dn
    else:
        print('computational error')
    return z, x1, x2

In [5]:
z, x1, x2 = integer(c1=100, c2=150, a11=3, a12=2, a21=1, a22=2, b1=6, b2=5)
print(f"Optimal solution: z={z}, x1={x1}, x2={x2}")

Optimal solution: z=300, x1=0, x2=2


# Part 3 Linear Programming solver execution

In [7]:
from ortools.linear_solver import pywraplp

In [8]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [9]:
infinity = solver.infinity()
w1c1 = solver.IntVar(0.0, infinity, 'w1c1')
w1c2 = solver.IntVar(0.0, infinity, 'w1c2')
w1c3 = solver.IntVar(0.0, infinity, 'w1c3')
w2c1 = solver.IntVar(0.0, infinity, 'w2c1')
w2c2 = solver.IntVar(0.0, infinity, 'w2c2')
w2c3 = solver.IntVar(0.0, infinity, 'w2c3')

print('Number of variables =', solver.NumVariables())

Number of variables = 6


In [10]:
#Demand Value
solver.Add(w1c1 + w2c1 == 600)

solver.Add(w1c2 + w2c2 == 700)

solver.Add(w1c3 + w2c3 == 300)

#Warehouse Capacity

solver.Add(w1c1 + w1c2 + w1c3 <= 500)

solver.Add(w2c1 + w2c2 + w2c3 <= 1800)

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 5


In [11]:
# Objective function
solver.Minimize(2*w1c1 + 1.5*w1c2 + 10*w1c3 + 4*w2c1 + 3.5*w2c2 + 6*w2c3)

In [12]:
# Now we run the Simplex algorithm to get the optimal solution
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('warehouse #1, City #1 =', w1c1.solution_value())
    print('warehouse #1, City #2 =', w1c2.solution_value())
    print('warehouse #1, City #3 =', w1c3.solution_value())
    print('warehouse #2, City #1 =', w2c1.solution_value())
    print('warehouse #2, City #2 =', w2c2.solution_value())
    print('warehouse #2, City #3 =', w2c3.solution_value())
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 5650.0
warehouse #1, City #1 = 0.0
warehouse #1, City #2 = 500.0
warehouse #1, City #3 = 0.0
warehouse #2, City #1 = 600.0
warehouse #2, City #2 = 200.0
warehouse #2, City #3 = 300.0
